# Comparison 

This file allows one to perform comparisons between the ZigZag approach and three methods for tracking:

1) Tracking using OpenCV algorithms by means of the MultiTracker class)
2) Tracking using OpenCV algorithms without using the MultiTracker class, but tracker instances individually for each detected object.
3) Classical approach based on detector + centroids

The comparison is performed using the folder "dataset" contained in the repository. For experiments 1) and 2) we use the different trackers available in OpenCV ("csrt", "kcf", "boosting", "mil", "medianflow", "mosse") as well as different detectors (findContours, Hough and findContours using the inner contours to detect the holes).

Please note that the experiments may need some hours to finish.

The ground truth file is the same as in the Experiments Jupyter notebook (ground_truth.txt) and contains the real number of connected components and holes (appearing in at least 20 frames) of the videos. This file will be used to compare the results. 

This experiments have been carried out in a laptop with an Intel Core i7-4810MQ 3.8GHz (4 cores, 8 threads) and 24GB RAM, running Ubuntu 20.04. To run the experiments, several package are required (`pandas`, `imutils`, `natsort`, `opencv-contrib-python`)

## Required imports

In [1]:
from imutils.video import VideoStream
from imutils.video import FPS
import argparse
import imutils
import time
import cv2
import os
from natsort import natsorted
import numpy as np
import csv
import shutil
import pandas as pd 

## Parameters 

With this cell we are selecting the parameters of the tracking

In [2]:
N = 20 # Number of consecutive frames required to mark an object as tracked.
FOLDER = r"dataset/" # Input folder of the different datasets.
OUTPUT_FOLDER = r"output_comparison" # Output folder for the results
JPG_EXPORT_QUALITY = 40

# Constant (a dictionary that maps strings to their corresponding 
# OpenCV object tracker implementations):
OPENCV_OBJECT_TRACKERS = {
    "csrt": cv2.legacy.TrackerCSRT_create,
    "kcf": cv2.legacy.TrackerKCF_create,
    "boosting": cv2.legacy.TrackerBoosting_create,
    "mil": cv2.legacy.TrackerMIL_create,    
    "medianflow": cv2.legacy.TrackerMedianFlow_create,
    "mosse": cv2.legacy.TrackerMOSSE_create,    
}

JPG_EXPORT_QUALITY = [int(cv2.IMWRITE_JPEG_QUALITY), JPG_EXPORT_QUALITY]

## Comparison 1: Tracking with openCV algorithms (MultiTracker class)

In [3]:
def process_video_multitracker(video_full_path, TRACKER, DETECTOR, verbose=0):
    time_start = time.time()
    imageFolder = video_full_path
    name = os.path.basename(video_full_path)    
    imageList = []
    for file_path in natsorted(os.listdir(imageFolder)):
        if os.path.isfile(os.path.join(imageFolder, file_path)):
            img = cv2.imread(os.path.join(imageFolder, file_path),0)
            if not img is None:                
                imageList.append(img)
                
    tracker = OPENCV_OBJECT_TRACKERS[TRACKER]
    
    # initialize OpenCV's special multi-object tracker
    trackers = cv2.legacy.MultiTracker_create()
    
    i = 0
    detected_objects = 0
    max_number_consecutive_success=0
    current_consecutive_success=0
    for frame in imageList:    
        frame_BGR = cv2.cvtColor(frame, cv2.COLOR_GRAY2BGR)    
        # If we are in the first frame, we extract the external contours to find the objects to track
        if i==0:               
            if DETECTOR == "findContours":
                # Only the exterior contours (connected components)
                contours, hierarchy = cv2.findContours(frame,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)[-2:]  
                idx=0 
                detected_objects = len(contours)
                for cnt in contours:
                    idx += 1
                    x,y,w,h = cv2.boundingRect(cnt)
                    trackers.add(tracker(), frame_BGR, [x,y,w,h])       
            elif DETECTOR == "Hough":
                circles = cv2.HoughCircles(frame, cv2.HOUGH_GRADIENT, 1, 8,
                               param1=300, param2=15,
                               minRadius=1, maxRadius=25)
                if circles is not None:
                    circles = np.uint16(np.around(circles))
                    detected_objects = len(circles[0,:])
                    for c in circles[0,:]:
                        center = (c[0], c[1])
                        radius = c[2]
                        x = c[0] - radius
                        y = c[1] - radius
                        h = 2*radius
                        w = 2*radius
                        trackers.add(tracker(), frame_BGR, [x,y,w,h])       
                    # To draw the circles:
                    # for c in circles[0, :]:
                    #     center = (c[0], c[1])
                    #      # circle center
                    #     cv2.circle(frame_BGR, center, 1, (0, 100, 100), 1)
                    #      # circle outline
                    #     radius = c[2]
                    #     cv2.circle(frame_BGR, center, radius, (255, 0, 255), 2)
            elif DETECTOR == "findContours_holes":
                # Only the interior contours (holes)
                contours, hierarchy = cv2.findContours(frame,cv2.RETR_CCOMP,cv2.CHAIN_APPROX_SIMPLE)[-2:]  
                holes = [contours[i] for i in range(len(contours)) if hierarchy[0][i][3] >= 0]
                idx=0 
                detected_objects = len(holes)
                for cnt in holes:
                    idx += 1
                    x,y,w,h = cv2.boundingRect(cnt)
                    trackers.add(tracker(), frame_BGR, [x,y,w,h])       
            else:
                raise("Error, no suitable detector selected")
        
        # Grab the new bounding box coordinates of the objects
        # When using MultiTracker, if any object is not detected in a frame then this line returns False
        (success, boxes) = trackers.update(frame_BGR)
        # check to see if the tracking was a success    
        if success:
            current_consecutive_success+=1
            if current_consecutive_success>max_number_consecutive_success:
                max_number_consecutive_success = current_consecutive_success
            # When using MultiTracker, if sucess, then the length of boxes will be equal to the number of detected objects.
            for box in boxes:                
                (x, y, w, h) = [int(v) for v in box]
                frame_BGR = cv2.rectangle(frame_BGR, (x, y), (x + w, y + h),(0, 255, 0), 2)
        else:
            if verbose == 1:
                print("Failed in frame", i)
            current_consecutive_success=0
        cv2.imwrite(OUTPUT_FOLDER+"/multitrack/"+name+"/"+DETECTOR+"/"+TRACKER+"/output_frame"+str(i)+".jpg", frame_BGR, JPG_EXPORT_QUALITY)
        i+=1

    #If we do not reach the threshold (at least N consecutive frames), then no objects have been detected.
    if max_number_consecutive_success < N:
        detected_objects = 0
    time_elapsed = time.time() - time_start    
    print("Detected", detected_objects, "objects in video", name, "using", DETECTOR, "and", TRACKER)
    line_to_write_csv = [name, DETECTOR, TRACKER, time_elapsed, detected_objects]
    return line_to_write_csv

In [4]:
def comparison_multitrack():
    detectors = ["Hough", "findContours", "findContours_holes"]
    tracking_algorithms = ["csrt", "kcf", "boosting", "mil", "medianflow", "mosse"]
    
    if not os.path.exists(OUTPUT_FOLDER):
        os.makedirs(OUTPUT_FOLDER)

    # open the csv file in the write mode
    file = open(os.path.join(OUTPUT_FOLDER,"results_multitrack.csv"), 'w')
    # create the csv writer
    writer = csv.writer(file)
    writer.writerow(["name", "detector", "tracker", "time", "detected_objects"])
    for file_path in natsorted(os.listdir(FOLDER)):
            # check if current file_path is a dir
            if (os.path.isdir(os.path.join(FOLDER, file_path)) and (not ("checkpoints" in file_path))):           
                print("Processing folder", file_path)
                for d in detectors:
                    for t in tracking_algorithms:
                        # First we create the corresponding output folder:
                        output_path = OUTPUT_FOLDER+"/multitrack/"+file_path+"/"+d+"/"+t
                        if not os.path.exists(output_path):
                            os.makedirs(output_path)
                        #Process the video
                        results = process_video_multitracker(FOLDER + "/"+file_path, TRACKER=t, DETECTOR=d)
                        writer.writerow(results)
                        file.flush()            
    # close the file
    file.close()    

comparison_multitrack()

Processing folder acA1920-155um__22949301__20200710_094124535
Detected 15 objects in video acA1920-155um__22949301__20200710_094124535 using Hough and csrt
Detected 15 objects in video acA1920-155um__22949301__20200710_094124535 using Hough and kcf
Detected 15 objects in video acA1920-155um__22949301__20200710_094124535 using Hough and boosting
Detected 15 objects in video acA1920-155um__22949301__20200710_094124535 using Hough and mil
Detected 15 objects in video acA1920-155um__22949301__20200710_094124535 using Hough and medianflow
Detected 0 objects in video acA1920-155um__22949301__20200710_094124535 using Hough and mosse
Detected 18 objects in video acA1920-155um__22949301__20200710_094124535 using findContours and csrt
Detected 18 objects in video acA1920-155um__22949301__20200710_094124535 using findContours and kcf
Detected 18 objects in video acA1920-155um__22949301__20200710_094124535 using findContours and boosting
Detected 18 objects in video acA1920-155um__22949301__202007

Detected 13 objects in video casabee3 using Hough and kcf
Detected 13 objects in video casabee3 using Hough and boosting
Detected 13 objects in video casabee3 using Hough and mil
Detected 13 objects in video casabee3 using Hough and medianflow
Detected 0 objects in video casabee3 using Hough and mosse
Detected 13 objects in video casabee3 using findContours and csrt
Detected 13 objects in video casabee3 using findContours and kcf
Detected 13 objects in video casabee3 using findContours and boosting
Detected 13 objects in video casabee3 using findContours and mil
Detected 13 objects in video casabee3 using findContours and medianflow
Detected 0 objects in video casabee3 using findContours and mosse
Detected 13 objects in video casabee3 using findContours_holes and csrt
Detected 13 objects in video casabee3 using findContours_holes and kcf
Detected 13 objects in video casabee3 using findContours_holes and boosting
Detected 13 objects in video casabee3 using findContours_holes and mil
Det

Detected 2 objects in video circles_and_squares3 using findContours and kcf
Detected 2 objects in video circles_and_squares3 using findContours and boosting
Detected 2 objects in video circles_and_squares3 using findContours and mil
Detected 0 objects in video circles_and_squares3 using findContours and medianflow
Detected 0 objects in video circles_and_squares3 using findContours and mosse
Detected 2 objects in video circles_and_squares3 using findContours_holes and csrt
Detected 2 objects in video circles_and_squares3 using findContours_holes and kcf
Detected 2 objects in video circles_and_squares3 using findContours_holes and boosting
Detected 2 objects in video circles_and_squares3 using findContours_holes and mil
Detected 2 objects in video circles_and_squares3 using findContours_holes and medianflow
Detected 2 objects in video circles_and_squares3 using findContours_holes and mosse
Processing folder different_shapes1
Detected 3 objects in video different_shapes1 using Hough and c

Detected 2 objects in video letters4 using findContours_holes and boosting
Detected 2 objects in video letters4 using findContours_holes and mil
Detected 2 objects in video letters4 using findContours_holes and medianflow
Detected 0 objects in video letters4 using findContours_holes and mosse
Processing folder letters5
Detected 7 objects in video letters5 using Hough and csrt
Detected 7 objects in video letters5 using Hough and kcf
Detected 7 objects in video letters5 using Hough and boosting
Detected 7 objects in video letters5 using Hough and mil
Detected 7 objects in video letters5 using Hough and medianflow
Detected 0 objects in video letters5 using Hough and mosse
Detected 12 objects in video letters5 using findContours and csrt
Detected 0 objects in video letters5 using findContours and kcf
Detected 12 objects in video letters5 using findContours and boosting
Detected 12 objects in video letters5 using findContours and mil
Detected 12 objects in video letters5 using findContours 

In [5]:
# OPTIONAL: To avoid thousands of files in the repository, now we zip all the output directory. Then, we delete it.
shutil.make_archive(OUTPUT_FOLDER+"/output_multitrack", 'zip', OUTPUT_FOLDER + "/multitrack/")
shutil.rmtree(OUTPUT_FOLDER + "/multitrack/")

## Comparison 2: Tracking with openCV algorithms (without MultipleTracker class)

In [6]:
def process_video_without_multitracker(video_full_path, TRACKER, DETECTOR, verbose=0):
    time_start = time.time()
    imageFolder = video_full_path
    name = os.path.basename(video_full_path)    
    imageList = []
    for file_path in natsorted(os.listdir(imageFolder)):
        if os.path.isfile(os.path.join(imageFolder, file_path)):
            img = cv2.imread(os.path.join(imageFolder, file_path),0)
            if not img is None:                
                imageList.append(img)
                
    tracker = OPENCV_OBJECT_TRACKERS[TRACKER]
    
    trackers = []    

    i = 0
    #detected_objects = []
    

    max_number_consecutive_success=0
    current_consecutive_success=0
    
    for frame in imageList:    
        frame_BGR = cv2.cvtColor(frame, cv2.COLOR_GRAY2BGR)    
        # If we are in the first frame, we extract the external contours to find the objects to track
        if i==0:               
            if DETECTOR == "findContours":
                # Only the exterior contours (connected components)
                contours, hierarchy = cv2.findContours(frame,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)[-2:]  
                idx=0 
                detected_objects = len(contours)
                for cnt in contours:
                    idx += 1
                    x,y,w,h = cv2.boundingRect(cnt)            
                    tracker2 = tracker()
                    trackers.append(tracker2)
                    tracker2.init(frame_BGR, [x,y,w,h])                    
            elif DETECTOR == "Hough":
                circles = cv2.HoughCircles(frame, cv2.HOUGH_GRADIENT, 1, 8,
                               param1=300, param2=15,
                               minRadius=1, maxRadius=25)
                if circles is not None:
                    circles = np.uint16(np.around(circles))
                    detected_objects = len(circles[0,:])
                    for c in circles[0,:]:
                        center = (c[0], c[1])
                        radius = c[2]
                        x = c[0] - radius
                        y = c[1] - radius
                        h = 2*radius
                        w = 2*radius
                        tracker2 = tracker()
                        trackers.append(tracker2)
                        tracker2.init(frame_BGR, [x,y,w,h])                        
                    # To draw the circles:
                    # for c in circles[0, :]:
                    #     center = (c[0], c[1])
                    #      # circle center
                    #     cv2.circle(frame_BGR, center, 1, (0, 100, 100), 1)
                    #      # circle outline
                    #     radius = c[2]
                    #     cv2.circle(frame_BGR, center, radius, (255, 0, 255), 2)
            elif DETECTOR == "findContours_holes":
                # Only the interior contours (holes)
                contours, hierarchy = cv2.findContours(frame,cv2.RETR_CCOMP,cv2.CHAIN_APPROX_SIMPLE)[-2:]  
                holes = [contours[i] for i in range(len(contours)) if hierarchy[0][i][3] >= 0]
                idx=0 
                detected_objects = len(holes)
                for cnt in holes:                                
                    x,y,w,h = cv2.boundingRect(cnt)            
                    tracker2 = tracker()
                    trackers.append(tracker2)
                    tracker2.init(frame_BGR, [x,y,w,h])
            else:
                raise("Error, no suitable detector selected")
            
            max_number_consecutive_success = np.zeros(detected_objects)
            current_consecutive_success = np.zeros(detected_objects)


        for idx, t in enumerate(trackers):
            (success, boxes) = t.update(frame_BGR)
            if success: 
                current_consecutive_success[idx] += 1
                if current_consecutive_success[idx] > max_number_consecutive_success[idx]:
                    max_number_consecutive_success[idx] = current_consecutive_success[idx]
                (x, y, w, h) = [int(v) for v in boxes]            
                frame_BGR = cv2.rectangle(frame_BGR, (x, y), (x + w, y + h),(0, 255, 0), 2)
            else:
                if verbose == 1:
                    print("Failed in frame", i)
                current_consecutive_success[idx]=0
        
        cv2.imwrite(OUTPUT_FOLDER+"/no_multitrack/"+name+"/"+DETECTOR+"/"+TRACKER+"/output_frame"+str(i)+".jpg", frame_BGR, JPG_EXPORT_QUALITY)
        i+=1

    # We count the objects that have been detected at least N consecutive frames
    detected_objects = (max_number_consecutive_success > N).sum()
    time_elapsed = time.time() - time_start    
    print("Detected", detected_objects, "objects in video", name, "using", DETECTOR, "and", TRACKER)
    line_to_write_csv = [name, DETECTOR, TRACKER, time_elapsed, detected_objects]
    return line_to_write_csv

In [7]:
def comparison_without_multitrack():
    detectors = ["Hough", "findContours", "findContours_holes"]
    tracking_algorithms = ["csrt", "kcf", "boosting", "mil", "medianflow", "mosse"]
    
    if not os.path.exists(OUTPUT_FOLDER):
        os.makedirs(OUTPUT_FOLDER)

    # open the csv file in the write mode
    file = open(os.path.join(OUTPUT_FOLDER,"results_without_multitrack.csv"), 'w')
    # create the csv writer
    writer = csv.writer(file)
    writer.writerow(["name", "detector", "tracker", "time", "detected_objects"])
    for file_path in natsorted(os.listdir(FOLDER)):
            # check if current file_path is a dir
            if (os.path.isdir(os.path.join(FOLDER, file_path)) and (not ("checkpoints" in file_path))):           
                print("Processing folder", file_path)
                for d in detectors:
                    for t in tracking_algorithms:
                        # First we create the corresponding output folder:
                        output_path = OUTPUT_FOLDER+"/no_multitrack/"+file_path+"/"+d+"/"+t
                        if not os.path.exists(output_path):
                            os.makedirs(output_path)
                        #Process the video
                        results = process_video_without_multitracker(FOLDER + "/"+file_path, TRACKER=t, DETECTOR=d)
                        writer.writerow(results)
                        file.flush()            
    # close the file
    file.close()    

comparison_without_multitrack()

Processing folder acA1920-155um__22949301__20200710_094124535
Detected 15 objects in video acA1920-155um__22949301__20200710_094124535 using Hough and csrt
Detected 15 objects in video acA1920-155um__22949301__20200710_094124535 using Hough and kcf
Detected 15 objects in video acA1920-155um__22949301__20200710_094124535 using Hough and boosting
Detected 15 objects in video acA1920-155um__22949301__20200710_094124535 using Hough and mil
Detected 15 objects in video acA1920-155um__22949301__20200710_094124535 using Hough and medianflow
Detected 0 objects in video acA1920-155um__22949301__20200710_094124535 using Hough and mosse
Detected 18 objects in video acA1920-155um__22949301__20200710_094124535 using findContours and csrt
Detected 18 objects in video acA1920-155um__22949301__20200710_094124535 using findContours and kcf
Detected 18 objects in video acA1920-155um__22949301__20200710_094124535 using findContours and boosting
Detected 18 objects in video acA1920-155um__22949301__202007

Detected 13 objects in video casabee3 using Hough and kcf
Detected 13 objects in video casabee3 using Hough and boosting
Detected 13 objects in video casabee3 using Hough and mil
Detected 12 objects in video casabee3 using Hough and medianflow
Detected 0 objects in video casabee3 using Hough and mosse
Detected 13 objects in video casabee3 using findContours and csrt
Detected 13 objects in video casabee3 using findContours and kcf
Detected 13 objects in video casabee3 using findContours and boosting
Detected 13 objects in video casabee3 using findContours and mil
Detected 13 objects in video casabee3 using findContours and medianflow
Detected 0 objects in video casabee3 using findContours and mosse
Detected 13 objects in video casabee3 using findContours_holes and csrt
Detected 13 objects in video casabee3 using findContours_holes and kcf
Detected 13 objects in video casabee3 using findContours_holes and boosting
Detected 13 objects in video casabee3 using findContours_holes and mil
Det

Detected 2 objects in video circles_and_squares3 using findContours and kcf
Detected 2 objects in video circles_and_squares3 using findContours and boosting
Detected 2 objects in video circles_and_squares3 using findContours and mil
Detected 1 objects in video circles_and_squares3 using findContours and medianflow
Detected 1 objects in video circles_and_squares3 using findContours and mosse
Detected 2 objects in video circles_and_squares3 using findContours_holes and csrt
Detected 2 objects in video circles_and_squares3 using findContours_holes and kcf
Detected 2 objects in video circles_and_squares3 using findContours_holes and boosting
Detected 2 objects in video circles_and_squares3 using findContours_holes and mil
Detected 2 objects in video circles_and_squares3 using findContours_holes and medianflow
Detected 2 objects in video circles_and_squares3 using findContours_holes and mosse
Processing folder different_shapes1
Detected 3 objects in video different_shapes1 using Hough and c

Detected 2 objects in video letters4 using findContours_holes and boosting
Detected 2 objects in video letters4 using findContours_holes and mil
Detected 2 objects in video letters4 using findContours_holes and medianflow
Detected 0 objects in video letters4 using findContours_holes and mosse
Processing folder letters5
Detected 7 objects in video letters5 using Hough and csrt
Detected 7 objects in video letters5 using Hough and kcf
Detected 7 objects in video letters5 using Hough and boosting
Detected 7 objects in video letters5 using Hough and mil
Detected 7 objects in video letters5 using Hough and medianflow
Detected 2 objects in video letters5 using Hough and mosse
Detected 12 objects in video letters5 using findContours and csrt
Detected 11 objects in video letters5 using findContours and kcf
Detected 12 objects in video letters5 using findContours and boosting
Detected 12 objects in video letters5 using findContours and mil
Detected 12 objects in video letters5 using findContours

In [8]:
# OPTIONAL: To avoid thousands of files in the repository, now we zip all the output directory. Then, we delete it.
shutil.make_archive(OUTPUT_FOLDER+"/output_without_multitrack", 'zip', OUTPUT_FOLDER + "/no_multitrack/")
shutil.rmtree(OUTPUT_FOLDER + "/no_multitrack/")

## Comparison 3: Tracking using the centroids approach

In [9]:
# This cell is based on https://www.pyimagesearch.com/2018/07/23/simple-object-tracking-with-opencv/
# We modify the code to count the number of consecutive apparitions of each object.

# import the necessary packages
from scipy.spatial import distance as dist
from collections import OrderedDict
import numpy as np

class CentroidTracker():
    def __init__(self, maxDisappeared=20):
        self.nextObjectID = 0
        self.objects = OrderedDict()
        self.disappeared = OrderedDict()
        self.max_consecutive_apparitions = OrderedDict() # Max consecutive apparitions of each object
        self.current_consecutive_apparitions = OrderedDict() # Current consecutive apparitions of each object
        self.maxDisappeared = maxDisappeared

    def register(self, centroid):
        self.objects[self.nextObjectID] = centroid
        self.disappeared[self.nextObjectID] = 0
        self.max_consecutive_apparitions[self.nextObjectID] = 0
        self.current_consecutive_apparitions[self.nextObjectID] = 0
        self.nextObjectID += 1

    def deregister(self, objectID):
        del self.objects[objectID]
        del self.disappeared[objectID]
        del self.max_consecutive_apparitions[objectID]
        del self.current_consecutive_apparitions[objectID]

    def update(self, rects):
        if len(rects) == 0:
            for objectID in self.disappeared.keys():
                self.disappeared[objectID] += 1
                self.current_consecutive_apparitions[objectID] = 0                 
                if self.disappeared[objectID] > self.maxDisappeared:
                    self.deregister(objectID)

            return self.objects

        inputCentroids = np.zeros((len(rects), 2), dtype="int")
        for (i, (startX, startY, endX, endY)) in enumerate(rects):
            cX = int((startX + endX) / 2.0)
            cY = int((startY + endY) / 2.0)
            inputCentroids[i] = (cX, cY)

        if len(self.objects) == 0:
            for i in range(0, len(inputCentroids)):
                self.register(inputCentroids[i])                

        else:
            objectIDs = list(self.objects.keys())
            objectCentroids = list(self.objects.values())
            D = dist.cdist(np.array(objectCentroids), inputCentroids)
            rows = D.min(axis=1).argsort()
            cols = D.argmin(axis=1)[rows]

            usedRows = set()
            usedCols = set()

            for (row, col) in zip(rows, cols):
                if row in usedRows or col in usedCols:
                    continue

                objectID = objectIDs[row]
                self.objects[objectID] = inputCentroids[col]
                self.disappeared[objectID] = 0
                self.current_consecutive_apparitions[objectID] += 1
                if self.current_consecutive_apparitions[objectID] > self.max_consecutive_apparitions[objectID]:
                    self.max_consecutive_apparitions[objectID] = self.current_consecutive_apparitions[objectID]
                usedRows.add(row)
                usedCols.add(col)
            unusedRows = set(range(0, D.shape[0])).difference(usedRows)
            unusedCols = set(range(0, D.shape[1])).difference(usedCols)
            if D.shape[0] >= D.shape[1]:
                for row in unusedRows:
                    objectID = objectIDs[row]
                    self.disappeared[objectID] += 1

                    if self.disappeared[objectID] > self.maxDisappeared:
                        self.deregister(objectID)
            else:
                for col in unusedCols:
                    self.register(inputCentroids[col])

        return self.objects

In [10]:
def process_video_centroids(video_full_path, DETECTOR, verbose=0):
    time_start = time.time()
    imageFolder = video_full_path
    name = os.path.basename(video_full_path)    
    imageList = []
    for file_path in natsorted(os.listdir(imageFolder)):
        if os.path.isfile(os.path.join(imageFolder, file_path)):
            img = cv2.imread(os.path.join(imageFolder, file_path),0)
            if not img is None:                
                imageList.append(img)
                
    
    ct = CentroidTracker()
    
    current_boxes=[]
    
    max_number_consecutive_success=0
    current_consecutive_success=0
    i=0
    for frame in imageList:    
        frame_BGR = cv2.cvtColor(frame, cv2.COLOR_GRAY2BGR)    
        # If we are in the first frame, we extract the external contours to find the objects to track
        #if i==0:   
        current_boxes=[]
        if DETECTOR == "findContours":
            # Only the exterior contours (connected components)
            contours, hierarchy = cv2.findContours(frame,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)[-2:]  
            idx=0 
            detected_objects = len(contours)
            for cnt in contours:
                idx += 1
                x,y,w,h = cv2.boundingRect(cnt)                                
                current_boxes.append((x,y,x+w,y+h)) # (startX, startY, endX, endY)
        elif DETECTOR == "Hough":
            circles = cv2.HoughCircles(frame, cv2.HOUGH_GRADIENT, 1, 8,
                           param1=300, param2=15,
                           minRadius=1, maxRadius=25)
            if circles is not None:
                circles = np.uint16(np.around(circles))
                detected_objects = len(circles[0,:])
                for c in circles[0,:]:
                    center = (c[0], c[1])
                    radius = c[2]
                    x = c[0] - radius
                    y = c[1] - radius
                    h = 2*radius
                    w = 2*radius
                    current_boxes.append((x,y,x+w,y+h))

        elif DETECTOR == "findContours_holes":
            # Only the interior contours (holes)
            contours, hierarchy = cv2.findContours(frame,cv2.RETR_CCOMP,cv2.CHAIN_APPROX_SIMPLE)[-2:]  
            holes = [contours[i] for i in range(len(contours)) if hierarchy[0][i][3] >= 0]
            idx=0 
            detected_objects = len(holes)
            for cnt in holes:                                
                x,y,w,h = cv2.boundingRect(cnt)            
                current_boxes.append((x,y,x+w,y+h))
        else:
            raise("Error, no suitable detector selected")
            
            max_number_consecutive_success = np.zeros(detected_objects)
            current_consecutive_success = np.zeros(detected_objects)
    
        objects = ct.update(current_boxes)

        for (objectID, centroid) in objects.items():
            text = "ID {}".format(objectID)
            #cv2.putText(frame, text, (centroid[0] - 10, centroid[1] - 10),
            #    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
            #cv2.rectangle(frame_BGR, (x, y), (x + w, y + h),(0, 255, 0), 2)
            cv2.circle(frame_BGR, (centroid[0], centroid[1]), 4, (0, 255, 0), -1)
        # show the output frame
        cv2.imwrite(OUTPUT_FOLDER+"/centroids/"+name+"/"+DETECTOR+"/output_frame"+str(i)+".jpg", frame_BGR, JPG_EXPORT_QUALITY)
        i+=1
        
    # We count the objects that have been detected at least N consecutive frames
    max_consecutive_apparitions = np.fromiter(ct.max_consecutive_apparitions.values(), dtype=int)
    detected_objects = (max_consecutive_apparitions > N).sum()
    time_elapsed = time.time() - time_start    
    print("Detected", detected_objects, "objects in video", name, "using", DETECTOR)
    line_to_write_csv = [name, DETECTOR, time_elapsed, detected_objects]
    return line_to_write_csv


In [11]:
def comparison_centroids():
    detectors = ["Hough", "findContours", "findContours_holes"]
    
    if not os.path.exists(OUTPUT_FOLDER):
        os.makedirs(OUTPUT_FOLDER)

    # open the csv file in the write mode
    file = open(os.path.join(OUTPUT_FOLDER,"results_centroids.csv"), 'w')
    # create the csv writer
    writer = csv.writer(file)
    writer.writerow(["name", "detector", "time", "detected_objects"])
    for file_path in natsorted(os.listdir(FOLDER)):
            # check if current file_path is a dir
            if (os.path.isdir(os.path.join(FOLDER, file_path)) and (not ("checkpoints" in file_path))):           
                print("Processing folder", file_path)
                for d in detectors:
                    # First we create the corresponding output folder:
                    output_path = OUTPUT_FOLDER+"/centroids/"+file_path+"/"+d+"/"
                    if not os.path.exists(output_path):
                        os.makedirs(output_path)
                    #Process the video
                    results = process_video_centroids(FOLDER + "/"+file_path, DETECTOR=d)
                    writer.writerow(results)
                    file.flush()            
    # close the file
    file.close()   

comparison_centroids()

Processing folder acA1920-155um__22949301__20200710_094124535
Detected 16 objects in video acA1920-155um__22949301__20200710_094124535 using Hough
Detected 18 objects in video acA1920-155um__22949301__20200710_094124535 using findContours
Detected 21 objects in video acA1920-155um__22949301__20200710_094124535 using findContours_holes
Processing folder acA1920-155um__22949301__20210628_091847394
Detected 20 objects in video acA1920-155um__22949301__20210628_091847394 using Hough
Detected 18 objects in video acA1920-155um__22949301__20210628_091847394 using findContours


/tmp/ipykernel_18955/2485433691.py:44: RuntimeWarning: overflow encountered in scalar subtract
  x = c[0] - radius
/tmp/ipykernel_18955/2485433691.py:45: RuntimeWarning: overflow encountered in scalar subtract
  y = c[1] - radius


Detected 27 objects in video acA1920-155um__22949301__20210628_091847394 using findContours_holes
Processing folder acA1920-155um__22949301__20210628_114754591
Detected 24 objects in video acA1920-155um__22949301__20210628_114754591 using Hough
Detected 22 objects in video acA1920-155um__22949301__20210628_114754591 using findContours
Detected 29 objects in video acA1920-155um__22949301__20210628_114754591 using findContours_holes
Processing folder casabee1
Detected 26 objects in video casabee1 using Hough
Detected 26 objects in video casabee1 using findContours
Detected 28 objects in video casabee1 using findContours_holes
Processing folder casabee2
Detected 16 objects in video casabee2 using Hough
Detected 12 objects in video casabee2 using findContours
Detected 18 objects in video casabee2 using findContours_holes
Processing folder casabee3
Detected 12 objects in video casabee3 using Hough
Detected 13 objects in video casabee3 using findContours
Detected 12 objects in video casabee3

In [12]:
# OPTIONAL: To avoid thousands of files in the repository, now we zip all the output directory. Then, we delete it.
shutil.make_archive(OUTPUT_FOLDER+"/output_centroids", 'zip', OUTPUT_FOLDER + "/centroids/")
shutil.rmtree(OUTPUT_FOLDER + "/centroids/")

## Comparison to the ground truth
Now we compare the results to the ground truth for each method. 

First, we load the ground truth file.

In [13]:
# Read the ground truth
df_ground_truth = pd.read_csv("ground_truth.txt")
total_H0_truth = df_ground_truth["H0"].sum()
total_H1_truth = df_ground_truth["H1"].sum()
df_ground_truth.head()

# Definition of detectors and tracking algorithms
detectors = ["Hough", "findContours", "findContours_holes"]
tracking_algorithms = ["csrt", "kcf", "boosting", "mil", "medianflow", "mosse"]

# Read the results of each experiment
df_multitrack = pd.read_csv(OUTPUT_FOLDER+"/results_multitrack.csv")
df_without_multitrack = pd.read_csv(OUTPUT_FOLDER+"/results_without_multitrack.csv")
df_centroids = pd.read_csv(OUTPUT_FOLDER+"/results_centroids.csv")

Note that the Hough circle transform can be used as object detector, but it only makes sense for detecting holes (H1) since some of the videos that belongs to CASABee have circles that merge together, thus the classical trackers will have problems and will produce many false negatives and false positives (see for instance the image ```acA1920-155um__22949301__20210628_091847394/Hough/csrt/output_frame0.jpg``` in the ```output_without_multitrack.zip``` file).

Note that baseline methods can sometimes produce more H0 components than the ground truth...but this is somehow correct. This is simply because sometimes the objects are separated in the first frame and a few frames later they are merged, then the zigzag method track them merged (if they appear merged for more than 20 frames), but the baseline methods track them separately. Since the baseline methods can be manually adapted to track them merged in such cases, we just count them as correct. Note also that Hough method can produce false positives in H1.

In [14]:
for df_baseline in [df_multitrack, df_without_multitrack, df_centroids]:
    for index, row in df_ground_truth.iterrows():  
        folder = row['folder']
        H0 = row['H0']
        H1 = row['H1']
        for index2, row2 in df_baseline.iterrows(): 
            if row2['name']==folder:
                if row2['detector'] == "findContours" and row2['detected_objects']>H0:
                    df_baseline.loc[index2, 'detected_objects'] = H0                                    
                elif row2['detector'] == "Hough" and row2['detected_objects']>H1:
                    df_baseline.loc[index2, 'detected_objects'] = H1
                elif row2['detector'] == "findContours_holes" and row2['detected_objects']>H1:
                    df_baseline.loc[index2, 'detected_objects'] = H1

### Method 1: OpenCV tracking algorithms using the MultiTracker class

In [15]:
df_multitrack.head()

,name,detector,tracker,time,detected_objects
0,acA1920-155um__22949301__20200710_094124535,Hough,csrt,14.055244,15
1,acA1920-155um__22949301__20200710_094124535,Hough,kcf,0.291438,15
2,acA1920-155um__22949301__20200710_094124535,Hough,boosting,9.304420,15
3,acA1920-155um__22949301__20200710_094124535,Hough,mil,36.748147,15
4,acA1920-155um__22949301__20200710_094124535,Hough,medianflow,0.747521,15


In [16]:
for d in detectors:
    df = df_multitrack.loc[df_multitrack['detector'] == d]
    total_time = df["time"].sum()
    print("Total time tracking without multitrack and with detector", d, "is", total_time, "seconds")

print("-------------------------------------------------")

for d in detectors:
    df = df_multitrack.loc[df_multitrack['detector'] == d]
    accuracy_H0_list = []
    accuracy_H1_list = []
    for t in tracking_algorithms:
        df_t = df.loc[df["tracker"] == t]
        detected_objects = df_t["detected_objects"].sum()
        accuracy_H0 = detected_objects/total_H0_truth
        accuracy_H1 = detected_objects/total_H1_truth
        if d == "Hough":
#             print("Total H0 detected using", d, "and", t, "is", detected_objects)
#             print("Accuracy H0 using", d, "and", t, "is", accuracy_H0)
            print("Total H1 detected using", d, "and", t, "is", detected_objects)
            print("Accuracy H1 using", d, "and", t, "is", accuracy_H1)
            # Due to the poor results, we exclude the Mosse tracker algorithm in the min-mean-max computation
            if t != "mosse":
                accuracy_H0_list.append(accuracy_H0)
                accuracy_H1_list.append(accuracy_H1)
        elif d == "findContours":
            print("Total H0 detected using", d, "and", t, "is", detected_objects)
            print("Accuracy H0 using", d, "and", t, "is", accuracy_H0)
            if t != "mosse":
                accuracy_H0_list.append(accuracy_H0)                        
        elif d == "findContours_holes":
            print("Total H1 detected using", d, "and", t, "is", detected_objects)
            print("Accuracy H1 using", d, "and", t, "is", accuracy_H1)      
            if t != "mosse":                
                accuracy_H1_list.append(accuracy_H1)
    print("**************************************************")
    if d == "Hough":
        # print("Min accuracy H0 +" ,d, min(accuracy_H0_list))
        # print("Mean accuracy H0 +" ,d, sum(accuracy_H0_list) / len(accuracy_H0_list))
        # print("Max accuracy H0 +" ,d, max(accuracy_H0_list))
        print("Min accuracy H1 +" ,d, min(accuracy_H1_list))
        print("Mean accuracy H1 +" ,d, sum(accuracy_H1_list) / len(accuracy_H1_list))
        print("Max accuracy H1 +" ,d, max(accuracy_H1_list))
    elif d == "findContours":
        print("Min accuracy H0 +" ,d, min(accuracy_H0_list))
        print("Mean accuracy H0 +" ,d, sum(accuracy_H0_list) / len(accuracy_H0_list))
        print("Max accuracy H0 +" ,d, max(accuracy_H0_list))        
    elif d == "findContours_holes":        
        print("Min accuracy H1 +" ,d, min(accuracy_H1_list))
        print("Mean accuracy H1 +" ,d, sum(accuracy_H1_list) / len(accuracy_H1_list))
        print("Max accuracy H1 +" ,d, max(accuracy_H1_list))
    print("**************************************************")


Total time tracking without multitrack and with detector Hough is 663.9560680389404 seconds
Total time tracking without multitrack and with detector findContours is 890.8916666507721 seconds
Total time tracking without multitrack and with detector findContours_holes is 862.4284751415253 seconds
-------------------------------------------------
Total H1 detected using Hough and csrt is 156
Accuracy H1 using Hough and csrt is 0.7647058823529411
Total H1 detected using Hough and kcf is 134
Accuracy H1 using Hough and kcf is 0.6568627450980392
Total H1 detected using Hough and boosting is 156
Accuracy H1 using Hough and boosting is 0.7647058823529411
Total H1 detected using Hough and mil is 156
Accuracy H1 using Hough and mil is 0.7647058823529411
Total H1 detected using Hough and medianflow is 156
Accuracy H1 using Hough and medianflow is 0.7647058823529411
Total H1 detected using Hough and mosse is 5
Accuracy H1 using Hough and mosse is 0.024509803921568627
******************************

### Method 2: OpenCV tracking algorithms without the MultiTracker class

In [17]:
df_without_multitrack.head()

,name,detector,tracker,time,detected_objects
0,acA1920-155um__22949301__20200710_094124535,Hough,csrt,13.610179,15
1,acA1920-155um__22949301__20200710_094124535,Hough,kcf,0.295604,15
2,acA1920-155um__22949301__20200710_094124535,Hough,boosting,8.206651,15
3,acA1920-155um__22949301__20200710_094124535,Hough,mil,36.387258,15
4,acA1920-155um__22949301__20200710_094124535,Hough,medianflow,0.693043,15


In [18]:
for d in detectors:
    df = df_without_multitrack.loc[df_without_multitrack['detector'] == d]
    total_time = df["time"].sum()
    print("Total time tracking without multitrack and with detector", d, "is", total_time, "seconds")
    
print("-------------------------------------------------")

for d in detectors:
    df = df_without_multitrack.loc[df_without_multitrack['detector'] == d]
    accuracy_H0_list = []
    accuracy_H1_list = []
    for t in tracking_algorithms:
        df_t = df.loc[df["tracker"] == t]
        detected_objects = df_t["detected_objects"].sum()
        accuracy_H0 = detected_objects/total_H0_truth
        accuracy_H1 = detected_objects/total_H1_truth
        if d == "Hough":
#             print("Total H0 detected using", d, "and", t, "is", detected_objects)
#             print("Accuracy H0 using", d, "and", t, "is", accuracy_H0)
            print("Total H1 detected using", d, "and", t, "is", detected_objects)
            print("Accuracy H1 using", d, "and", t, "is", accuracy_H1)
            # Due to the poor results, we exclude the Mosse tracker algorithm in the min-mean-max computation
            if t != "mosse":
                accuracy_H0_list.append(accuracy_H0)
                accuracy_H1_list.append(accuracy_H1)
        elif d == "findContours":
            print("Total H0 detected using", d, "and", t, "is", detected_objects)
            print("Accuracy H0 using", d, "and", t, "is", accuracy_H0)
            if t != "mosse":
                accuracy_H0_list.append(accuracy_H0)                        
        elif d == "findContours_holes":
            print("Total H1 detected using", d, "and", t, "is", detected_objects)
            print("Accuracy H1 using", d, "and", t, "is", accuracy_H1)      
            if t != "mosse":                
                accuracy_H1_list.append(accuracy_H1)
    print("**************************************************")
    if d == "Hough":
        #print("Min accuracy H0 +" ,d, min(accuracy_H0_list))
        #print("Mean accuracy H0 +" ,d, sum(accuracy_H0_list) / len(accuracy_H0_list))
        #print("Max accuracy H0 +" ,d, max(accuracy_H0_list))
        print("Min accuracy H1 +" ,d, min(accuracy_H1_list))
        print("Mean accuracy H1 +" ,d, sum(accuracy_H1_list) / len(accuracy_H1_list))
        print("Max accuracy H1 +" ,d, max(accuracy_H1_list))
    elif d == "findContours":
        print("Min accuracy H0 +" ,d, min(accuracy_H0_list))
        print("Mean accuracy H0 +" ,d, sum(accuracy_H0_list) / len(accuracy_H0_list))
        print("Max accuracy H0 +" ,d, max(accuracy_H0_list))        
    elif d == "findContours_holes":        
        print("Min accuracy H1 +" ,d, min(accuracy_H1_list))
        print("Mean accuracy H1 +" ,d, sum(accuracy_H1_list) / len(accuracy_H1_list))
        print("Max accuracy H1 +" ,d, max(accuracy_H1_list))
    print("**************************************************")


Total time tracking without multitrack and with detector Hough is 660.1634395122528 seconds
Total time tracking without multitrack and with detector findContours is 884.4668128490448 seconds
Total time tracking without multitrack and with detector findContours_holes is 859.1599345207214 seconds
-------------------------------------------------
Total H1 detected using Hough and csrt is 156
Accuracy H1 using Hough and csrt is 0.7647058823529411
Total H1 detected using Hough and kcf is 155
Accuracy H1 using Hough and kcf is 0.7598039215686274
Total H1 detected using Hough and boosting is 156
Accuracy H1 using Hough and boosting is 0.7647058823529411
Total H1 detected using Hough and mil is 156
Accuracy H1 using Hough and mil is 0.7647058823529411
Total H1 detected using Hough and medianflow is 152
Accuracy H1 using Hough and medianflow is 0.7450980392156863
Total H1 detected using Hough and mosse is 11
Accuracy H1 using Hough and mosse is 0.05392156862745098
******************************

### Method 3: detector (Hough and findContours) + centroid tracking

In [19]:
for d in detectors:
    df = df_centroids.loc[df_centroids['detector'] == d]
    total_time = df["time"].sum()
    print("Total time using centroids with detector", d, "is", total_time, "seconds")

Total time using centroids with detector Hough is 2.437615156173705 seconds
Total time using centroids with detector findContours is 1.2338390350341788 seconds
Total time using centroids with detector findContours_holes is 1.3464608192443837 seconds


In [20]:
# Centroids + Hough
df_Hough = df_centroids.loc[df_centroids['detector'] == "Hough"]
total_H0_Hough = df_Hough["detected_objects"].sum()
#print("Total H0 detected with centroids and Hough (note that could be here false positives!)", total_H0_Hough)
#print("Accuracy Centroids + Hough", total_H0_Hough/total_H0_truth)
total_H1_Hough = df_Hough["detected_objects"].sum()
print("Total H1 detected with centroids and Hough (note that there could be some false positives)", total_H1_Hough)
print("Upper bound accuracy H1 Centroids + Hough", total_H1_Hough/total_H1_truth)

Total H1 detected with centroids and Hough (note that there could be some false positives) 158
Upper bound accuracy H1 Centroids + Hough 0.7745098039215687


In [21]:
# Centroids + FindContours
df_findContours = df_centroids.loc[df_centroids['detector'] == "findContours"]
total_H0_findContours = df_findContours["detected_objects"].sum()
print("Total H0 detected with centroids and findContours", total_H0_findContours)
print("Accuracy Centroids + findContours", total_H0_findContours/total_H0_truth)
# For H1 (the holes), we use the findContours_holes
df_findContours = df_centroids.loc[df_centroids['detector'] == "findContours_holes"]
total_H1_findContours = df_findContours["detected_objects"].sum()
print("Total H1 detected with centroids and findContours_holes", total_H1_findContours)
print("Accuracy H1 Centroids + findContours_holes", total_H1_findContours/total_H1_truth)

Total H0 detected with centroids and findContours 184
Accuracy Centroids + findContours 0.9633507853403142
Total H1 detected with centroids and findContours_holes 197
Accuracy H1 Centroids + findContours_holes 0.9656862745098039
